In [1]:
import numpy as np
import matplotlib.pylab as plt
import os

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else "cpu"

### Parsing Dataset

In [3]:
%matplotlib inline

files = os.listdir('sv_traces/')

### reads an sv_trace file into 2 structures:
### pcs: a list of 1001 16-bit pc values
### mem: a list of 1001 65542-byte memory vectors
def read_trace(sv_file):
    with open(sv_file, 'rb') as f:
        lines = f.read()
        lines = np.frombuffer(lines, dtype=np.uint8)
        lines = lines.reshape(1001,-1)
        pcs = lines[:, 0:2]
        mem = lines[:, :]
        return mem, pcs
    
### reads trace data into a dataset of mem-to-pc mappings, where
### mem_current: partial memory trace &
### pcs_current: full pcs trace
def read_dataset(file):
    mem_trace = dict_traces[file][0]
    mem_current = mem_trace[:]
    pcs_trace = dict_traces[file][1]
    pcs_current = pcs_trace[:]
    return mem_current, pcs_current

### dict_traces[file][0]: memory trace of <file>
### dict_traces[file][1]: pc trace of <file>
dict_traces = {}
for file in files[0:20]:
    dict_traces[file] = read_trace(os.path.join('sv_traces/', file))
    
### dict_dataset[file][0]: memory trace of <file>
### dict_dataset[file][1]: pc trace of <file>
dict_dataset = {}
for trace in dict_traces:
    dict_dataset[trace] = read_dataset(trace)

### Defining Train and Test Data

In [4]:
features, labels = dict_dataset[list(dict_dataset.keys())[0]]
for file in list(dict_dataset.keys())[1:10]:
    features = np.append(features, dict_dataset[file][0], axis=0)
    labels = np.append(labels, dict_dataset[file][1], axis=0)

features_test, labels_test = dict_dataset[list(dict_dataset.keys())[10]]
for file in list(dict_dataset.keys())[11:13]:
    features_test = np.append(features_test, dict_dataset[file][0], axis=0)
    labels_test = np.append(labels_test, dict_dataset[file][1], axis=0)
    
partial_mem_trace = features[: , 0:600]
features = partial_mem_trace

partial_mem_trace_test = features_test[: , 0:600]
features_test = partial_mem_trace_test

    
print("# data points = ", features.shape[0])
print("# features = ", features.shape[1])

# data points =  10010
# features =  600


### Generating Synthetic Dataset
#### This will reflect how well the network learns indexing.

In [5]:
# generate N vectors of 600 bytes
# learn indexing into second byte
def gen_data(N=1000, d=600, low=0, high=127, target_idx=1):
    data = np.random.randint(low=low, high=high, size=(N,d))
    return data, data[:, target_idx]

In [6]:
data = gen_data()
#print(data)
features = data[0]
print(features[0:3])
labels = data[1]
print(labels[0:3])
test_data = gen_data(N=200)
features_test = test_data[0]
print(features_test[0:3])
labels_test = test_data[1]
print(labels_test[0:3])

[[  9  23  52 ...  88  16  37]
 [122  36 105 ...   2  67  81]
 [119  78  81 ...  10   1 119]]
[23 36 78]
[[89 80 99 ...  3 73 31]
 [58 73 13 ... 84  8  4]
 [11 60 42 ... 20 30 59]]
[80 73 60]


### Defining Conv. NN

In [7]:
net = nn.Sequential(nn.Conv2d(1, 8, kernel_size=(1,8), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(8, 16, kernel_size=(1,8), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(16, 32, kernel_size=(1,8), stride=1),
                    nn.ReLU(),
                    nn.Flatten(start_dim=1, end_dim=-1),
                   )

In [8]:
net(torch.from_numpy(features[0]).unsqueeze(0).unsqueeze(0).unsqueeze(0).float()).shape 

torch.Size([1, 18528])

In [9]:
net = nn.Sequential(nn.Conv2d(1, 8, kernel_size=(1,8), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(8, 16, kernel_size=(1,8), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(16, 32, kernel_size=(1,8), stride=1),
                    nn.ReLU(),
                    nn.Flatten(start_dim=1, end_dim=-1),
                    nn.Linear(18528, 1)
                   )

In [10]:
# error between real and predicted 2 16-bit values
criterion = nn.MSELoss()

In [11]:
optimizer = optim.Adam(net.parameters(), lr=1e-5)

### Training and Validation

In [12]:
def validate(net, test_data, test_target, net_type=None):
    net = net.eval()
    with torch.no_grad():
        test_data_torch = torch.from_numpy(test_data).float().to(device)
        test_target_torch = torch.from_numpy(test_target).float().to(device)

        if net_type=='conv':
            test_data_torch = test_data_torch.unsqueeze(1).unsqueeze(2)
        
        test_pred = np.rint(net(test_data_torch).detach().numpy())
        test_accuracy = (test_target_torch.detach().numpy() == test_pred).mean()

        test_rmse = (np.sqrt((test_pred-test_target_torch.detach().numpy())**2).mean())

    return test_accuracy, test_rmse

def train_net(net, train_data, train_target, test_data, test_target, N_epochs=10, print_freq=100, net_type=None):
    net = net.to(device)
    train_data_torch = torch.from_numpy(train_data).float().to(device)
    train_target_torch = torch.from_numpy(train_target).to(device)
    
    if net_type=='conv':
        train_data_torch = train_data_torch.unsqueeze(1).unsqueeze(2)

    net = net.train()
    for i in range(N_epochs):
        pred = net(train_data_torch.float())
        loss = criterion(pred, train_target_torch.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % print_freq == 0:
            train_accuracy, train_rmse = validate(net, train_data, train_target, net_type=net_type)
            test_accuracy, test_rmse = validate(net, test_data, test_target, net_type=net_type)
            net = net.train()
            pred = np.rint(pred.detach().numpy())
            print("pred: ", pred[i], "target: ", train_target[i])
            print(f'Epoch = {i} loss={loss} train_acc = {train_accuracy:.3f} test_acc = {test_accuracy:.3f} train_rmse = {train_rmse:.3f}  test_rmse = {test_rmse:.3f}')
            
    return net

In [14]:
net = train_net(net, features, labels, features_test, labels_test, N_epochs=500, print_freq=10, net_type='conv')

torch.save(net, "conv_0")

pred:  [65.] target:  23
Epoch = 0 loss=1325.8577880859375 train_acc = 0.007 test_acc = 0.005 train_rmse = 31.423  test_rmse = 29.041
pred:  [63.] target:  25
Epoch = 10 loss=1325.854736328125 train_acc = 0.007 test_acc = 0.005 train_rmse = 31.423  test_rmse = 29.041
pred:  [63.] target:  94
Epoch = 20 loss=1325.8516845703125 train_acc = 0.007 test_acc = 0.005 train_rmse = 31.423  test_rmse = 29.041
pred:  [65.] target:  54
Epoch = 30 loss=1325.8482666015625 train_acc = 0.007 test_acc = 0.005 train_rmse = 31.423  test_rmse = 29.041
pred:  [63.] target:  46
Epoch = 40 loss=1325.84521484375 train_acc = 0.007 test_acc = 0.005 train_rmse = 31.423  test_rmse = 29.041
pred:  [64.] target:  102
Epoch = 50 loss=1325.8419189453125 train_acc = 0.007 test_acc = 0.005 train_rmse = 31.423  test_rmse = 29.041
pred:  [65.] target:  83
Epoch = 60 loss=1325.838623046875 train_acc = 0.007 test_acc = 0.005 train_rmse = 31.423  test_rmse = 29.041
pred:  [58.] target:  70
Epoch = 70 loss=1325.8350830078125